In [1]:
import pandas as pd
import json
from tqdm import trange

In [2]:
def read_jsonl(file_path):
    result_list = []
    with open(file_path, 'r') as f:
        f.seek(0, 2)
        eof = f.tell()
        f.seek(0, 0)
        while f.tell() < eof:
            result_list.append(json.loads(f.readline()))
    return result_list

In [3]:
df_label = pd.read_csv("/run/media/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-negbio.csv")
df_dicom = pd.read_csv("/run/media/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-metadata.csv")
train_metadata_list = read_jsonl("/run/media/mimic-pa-512/train/metadata.jsonl")
valid_metadata_list = read_jsonl("/run/media/mimic-pa-512/valid/metadata.jsonl")
df_train_meta = pd.DataFrame(train_metadata_list)
df_valid_meta = pd.DataFrame(valid_metadata_list)

In [4]:
df_label = df_label.fillna(0)
df_label = df_label.astype('int32')
df_label.head()

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10000032,50414267,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,10000032,53189527,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,10000032,53911762,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,10000032,56699142,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,10000764,57375967,0,0,1,0,0,0,0,0,0,0,0,-1,0,0


In [5]:
df_dicom.head()

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN


In [6]:
df_valid_meta.head()

,file_name,impression
0,0fe0f450-979ab2eb-ec87fcd3-c1660ba9-18517d0a.jpg,Right mid lung rounded opacity may represent a...
1,e75dad5d-03a961c6-602a953c-9bb24d1f-aaa47f83.jpg,"No radiographic evidence of pneumonia, intrath..."
2,02f3df2f-ff2bc640-5f173dca-eaff305d-73b20ae1.jpg,No acute cardiopulmonary abnormality.
3,4a8eb791-9bebcee1-8b01dcce-835b9a48-5d8896da.jpg,No acute cardiopulmonary process.
4,737caa62-adda5c8a-ad024448-5d85cbab-ec10a535.jpg,No acute cardiopulmonary process.


In [7]:
df_valid_meta.describe()

,file_name,impression
count,7868,7868
unique,7868,4020
top,0fe0f450-979ab2eb-ec87fcd3-c1660ba9-18517d0a.jpg,No acute cardiopulmonary process.
freq,1,1444


In [11]:
def is_positive(value):
    if int(value) == 1:
        return True
    else:
        return False

def label_count(meta_df, df_dicom, df_label):
    dicom_list = meta_df['file_name'].to_list()
    for i in range(len(dicom_list)):
        dicom_list[i] = dicom_list[i].split('.')[0]
    label_list = df_label.columns.to_list()[2:]
    stat_dict = dict()
    multi_stat_dict = dict()
    stat_dict['total'] = 0
    multi_stat_dict['total'] = 0
    for item in label_list:
        stat_dict[item] = 0

    for i in trange(len(dicom_list)):
        dicom_id = dicom_list[i]
        _dicom_index = df_dicom[df_dicom['dicom_id']==dicom_id].index
        _dicom_row = df_dicom.loc[_dicom_index]
        subject_id = int(_dicom_row['subject_id'].iloc[0])
        study_id = int(_dicom_row['study_id'].iloc[0])
        _label_index = df_label[(df_label['study_id']==study_id) & (df_label['subject_id']==subject_id)].index
        row = df_label.loc[_label_index]

        positive_list = []

        for label in label_list:
            value = row[label].iloc[0]
            if is_positive(value):
                stat_dict[label] += 1
                positive_list.append(label)

        positive_list.sort()
        if len(positive_list)>=2:
            multi_stat_dict['total'] += 1
            for j in range(len(positive_list)):
                for n in range(j+1, len(positive_list)):
                    multi_positive_label = f"{positive_list[j]}-{positive_list[n]}"
                    if multi_positive_label in multi_stat_dict.keys():
                        multi_stat_dict[multi_positive_label] += 1
                    else:
                        multi_stat_dict[multi_positive_label] = 1

        stat_dict['total'] += 1
    return stat_dict, multi_stat_dict
    



In [12]:
valid_stat, valid_multi_stat = label_count(df_valid_meta, df_dicom, df_label)
valid_stat

100%|██████████| 7868/7868 [01:41<00:00, 77.27it/s]


{'total': 7868,
 'Atelectasis': 690,
 'Cardiomegaly': 569,
 'Consolidation': 80,
 'Edema': 223,
 'Enlarged Cardiomediastinum': 62,
 'Fracture': 76,
 'Lung Lesion': 150,
 'Lung Opacity': 896,
 'No Finding': 4684,
 'Pleural Effusion': 891,
 'Pleural Other': 55,
 'Pneumonia': 367,
 'Pneumothorax': 131,
 'Support Devices': 329}

In [15]:
valid_multi_stat

{'total': 1315,
 'Atelectasis-Cardiomegaly': 86,
 'Atelectasis-Fracture': 10,
 'Atelectasis-Pleural Effusion': 272,
 'Cardiomegaly-Fracture': 7,
 'Cardiomegaly-Pleural Effusion': 133,
 'Fracture-Pleural Effusion': 2,
 'Atelectasis-Lung Opacity': 196,
 'Pleural Effusion-Support Devices': 67,
 'Lung Lesion-Lung Opacity': 52,
 'Lung Lesion-Pneumonia': 5,
 'Lung Opacity-Pneumonia': 127,
 'Lung Opacity-Pneumothorax': 17,
 'Lung Opacity-Support Devices': 34,
 'Pneumothorax-Support Devices': 38,
 'Cardiomegaly-Lung Opacity': 57,
 'Cardiomegaly-Support Devices': 56,
 'Lung Opacity-Pleural Effusion': 180,
 'Consolidation-Pneumonia': 21,
 'Consolidation-Lung Opacity': 20,
 'Cardiomegaly-Edema': 72,
 'Lung Lesion-Support Devices': 16,
 'Edema-Pleural Effusion': 102,
 'No Finding-Support Devices': 113,
 'Edema-Lung Opacity': 52,
 'Cardiomegaly-Enlarged Cardiomediastinum': 2,
 'Enlarged Cardiomediastinum-Pleural Effusion': 11,
 'Atelectasis-Edema': 29,
 'Atelectasis-Support Devices': 51,
 'Cardiome

In [10]:
train_stat = label_count(df_train_meta, df_dicom, df_label)
train_stat

100%|██████████| 71075/71075 [15:08<00:00, 78.24it/s]


{'total': 71075,
 'Atelectasis': 5947,
 'Cardiomegaly': 4598,
 'Consolidation': 975,
 'Edema': 2058,
 'Enlarged Cardiomediastinum': 568,
 'Fracture': 781,
 'Lung Lesion': 1565,
 'Lung Opacity': 8021,
 'No Finding': 43251,
 'Pleural Effusion': 6684,
 'Pleural Other': 440,
 'Pneumonia': 3562,
 'Pneumothorax': 1135,
 'Support Devices': 2855}